# School Greening and Urban Forests
#### Code written by Cami Pawlak, Jessica Baiza,  Dr. Aaron Liggett, Dr. Andrew Fricker

### Import Packages

In [ ]:
from arcpy import *
from arcpy.ra import *
from arcpy.sa import *
from arcpy import conversion
from arcpy.conversion import ExportFeatures
from arcpy import env

## Set Environment and load any file paths

#### To run this script, you will need to update the environment path, if the data is downloaded from the repository, the rest of the paths should work.

In [ ]:
#Set your workspace
arcpy.env.workspace = r'SET YOUR PATH/for_script_2/'
workspace_path = r'SET YOUR PATH/for_script_2/'

In [ ]:
#Schools points 2021-2022
school_pts_edge = r"schools\schools_pts_more_accurate\Public_School_Locs\EDGE_ADMINDATA_PUBLICSCH.shp"

#School points 2020-2021
school_pts_att = r"schools\school_pts_attributed\California_Schools_2020-21.gdb\2020-21.gdb\SchoolSites2021"

#School polygons 2020/2021
CSCD_school_polys = r"schools\schools_polygons_CSCD\CSCD_2021\CSCD_2021.gdb\Primary_Secondary\Schools_Current_Stacked"

#School polygons 2018
school_polys_18 = r"schools\california_school_polygons_18\schools_current_stacked_18_2.shp"

#Data we create: joined school polygons
spatial_join_school_polys =r"schools\merged\joined_school_polygons_spatial.shp"

#Data we create: school polygons unique to 2018
unique_18_polys = r"schools\california_school_polygons_18\unique_schools_18.shp"

#Data we create: Merged school polygons from 2018 and 2020/2021
merged_school_polys = r"schools\merged\merged_school_polygons.shp"

#Data we create: Spatial join schools to polygons, without climate and CalEnviroscreen data
schools_final = r"schools\final\merged_school_polygons_joined_to_merged_points_sp_tab.shp"

#California buildings file
buildings = r"C:\Users\cpawlak\Cal Poly\G. Andrew Fricker - Baiza\data\for_script\buildings\CA_Buildings2.gdb\California_Polygons"

#Data we create: Merged school polygons with buildings erased
school_polys_erase_buildings = r"schools\final\merged_school_polygons_joined_to_merged_points_sp_tab_erase_buildings.shp"

#USFS Canopy Cover Map for California 2018
canopy = r"canopy_cover\CA_urbancanopy_2018.gdb\CA_urbancanopy_2018\Band_1"

#Data we create: The canopy cover sums in each school polygon
canopy_table = r"canopy_cover\canopy_18_zonal_stats.dbf"

#Cal Enviroscreen Shapefile
cal_enviro = r"calenviroscreen\calenviroscreen40shpf2021shp\calenviroscreen40shpf2021shp\CES4 Final Shapefile.shp"

#Folder which stores climate models from CalAdapt
climate_folder = r"climate_models"

#Data we create: Schools with all attribute data and joins
schools_final_all = r"schools\final\final.gdb\school_polys_all_data"

#Data we created and use in this script: Center points for all school polygons
points = r'schools\joins\schools_polygons_final_erased_buildings_as_points.shp'

#Data we created: School polygon center points joined to climate information
points = r'schools\joins\school_polygons_final_erased_buildings_as_points.shp'
out_points = r'schools\joins\school_polygons_final_erased_buildings_as_points_join.shp'

## Tabular (text) Join of points to points
#### This section joins our two sets of school points to combine their attribute data.

In [ ]:
#Join based on code, this is the unique school identifier
in_field = "NCESSCH" 

#join field
join_field = "FedID"


#run the join which will add directly to school_pts_edge
joined_school_points = arcpy.management.AddJoin(school_pts_edge, in_field,
                         school_pts_att, join_field, "KEEP_ALL")


## Join the two school polygons

#### First join them to get a join count

In [ ]:
#join one to one or one
join_operation = 'JOIN_ONE_TO_ONE'

#keep all or keep common features
join_type = 'KEEP_ALL'

#field mapping, ignore, '#'

#match option is within a distance for search radius
match_option ='INTERSECT'

arcpy.analysis.SpatialJoin(school_polys_18, CSCD_school_polys, 
                           spatial_join_school_polys, 
                           join_operation, join_type,
                           '#', 
                           match_option)

#### Export just the unique polygons from 2018 schools

In [ ]:
arcpy.conversion.ExportFeatures(spatial_join_school_polys, unique_18_polys,'Join_Count = 0')

#### Merge the 2021 school polygons and unique 2018 schools

In [ ]:
arcpy.management.Merge([CSCD_school_polys, unique_18_polys], merged_school_polys)


#### Add Join any and all school information to the polygons (tabular)

In [ ]:

#Join based on code, this is the unique school identifier
in_field = "CDSCode" 

#join field
join_field = "SchoolSites2021.CDSCode"


#run the join which will add directly to school_pts_edge
school_polys_to_points_1 = arcpy.management.AddJoin(merged_school_polys, in_field,
                         joined_school_points, join_field, "KEEP_ALL")

#### Now spatial join with a 30 meter search radius

In [ ]:
#join one to one
join_operation = 'JOIN_ONE_TO_ONE'
#keep all or keep common features
join_type = 'KEEP_ALL'
#field mapping, ignore, '#'
#match option is within a distance for search radius
match_option ='WITHIN A DISTANCE'

search_radius = 30

schools_final = arcpy.analysis.SpatialJoin(school_polys_to_points_1, joined_school_points, 
                           schools_final, 
                           join_operation, join_type,
                           '#', 
                           match_option, search_radius)

## Erase buildings
#### This way, we can calculate canopy cover area without including buildings, which are not accessible
#### The schoolbuilding file linked only includes building  in California schools,  which we pre-clipped

In [ ]:
arcpy.analysis.PairwiseErase(schools_final, buildings, school_polys_erase_buildings)

## Summarize rasters by polygons

#### Get the canopy areas within each school polygon. Because our canopy pixel resolution is 1x1 m, the SUM statistic represents area

In [ ]:

#zone field (something unique)
zone_field = 'FID'

#statistics type, for canopy we want SUM, outputs meters squared of canopy
statistics_type = 'SUM'

#running zonal stats using defaults of ignoring no data and interpolation
arcpy.sa.ZonalStatisticsAsTable(school_polys_erase_buildings, zone_field, canopy, 
                                canopy_table,'DATA', statistics_type)

## Add climate models and get predicted climate futures for schools

In [ ]:
import os
os.chdir(workspace_path)
folder_path = r"climate_models/"  # Replace with the path to your folder
file_names = [file for file in os.listdir(folder_path) if file.endswith(".tif")]

In [ ]:
for climate_raster in file_names:
    
    #file_name = os.path.basename(file_name)
    climate_raster_path = folder_path+climate_raster
    
    # Set the field name to the raster name without the extension
    field_name =climate_raster.split(".")[0]
    
    # Use the ExtractValuesToPoints tool to extract raster values to points
    #these points were created by us, they are center points of the school polys(points)
    arcpy.sa.ExtractValuesToPoints(points, climate_raster_path, out_points, "NONE", "VALUE_ONLY")
    
    # Add a new field with the raster name as the field name
    arcpy.AddField_management(out_points, field_name, "DOUBLE")
    
    # Calculate the raster values to the new field
    arcpy.CalculateField_management(out_points, field_name, "!RASTERVALU!", "PYTHON3")
    
    out_climate_points = folder_path+field_name+".shp"
    #save it as a point file with climate info
    arcpy.conversion.ExportFeatures(out_points, out_climate_points)
    

#### Join tables
##### First join  canopy   values to school polygons file (erased buildings)

In [ ]:

#Join based on code, this is the unique school identifier
in_field = "FID" 
#join field
join_field = "FID_"

#run the join which will add directly to school_pts_edge
canopy_join = arcpy.management.AddJoin(school_polys_erase_buildings, in_field,
                         canopy_table, join_field, "KEEP_ALL")

canopy_join_path = r"join/schools_buildings_erased_canopy.shp"

#save it
arcpy.conversion.ExportFeatures(canopy_join, canopy_join_path)
    

#### Now join each climate point file for each climate scenario to the school polygons with buildings erased

In [ ]:
#Join based on code, this is the unique school identifier
in_field = "FID" 
#join field
join_field = "FID"
clim_join_path_1 = r"join/schools_buildings_erased_canopy_clim1.shp"
#run the join which will add directly to school_pts_edge
clim_join = arcpy.management.AddJoin(canopy_join_path, in_field,
                                     r"climate_models\avg30y_h.shp",
                                     join_field, "KEEP_ALL")

arcpy.conversion.ExportFeatures(clim_join, clim_join_path_1)
 

#Join based on code, this is the unique school identifier
in_field = "FID" 
#join field
join_field = "FID"
clim_join_path_2 = r"join/schools_buildings_erased_canopy_clim2.shp"
#run the join which will add directly to school_pts_edge
clim_join = arcpy.management.AddJoin(clim_join_path_1, in_field,
                                     r"climate_models\rcp45_fut.shp",
                                     join_field, "KEEP_ALL")

arcpy.conversion.ExportFeatures(clim_join, clim_join_path_2)



#Join based on code, this is the unique school identifier
in_field = "FID" 
#join field
join_field = "FID"
clim_join_path_3 = r"join/schools_buildings_erased_canopy_clim3.shp"
#run the join which will add directly to school_pts_edge
clim_join = arcpy.management.AddJoin(clim_join_path_2, in_field,
                                     r"climate_models\rcp45_mid.shp",
                                     join_field, "KEEP_ALL")

arcpy.conversion.ExportFeatures(clim_join, clim_join_path_3)


#Join based on code, this is the unique school identifier
in_field = "FID" 
#join field
join_field = "FID"
clim_join_path_4 = r"join/schools_buildings_erased_canopy_clim4.shp"
#run the join which will add directly to school_pts_edge
clim_join = arcpy.management.AddJoin(clim_join_path_3, in_field,
                                     r"climate_models\rcp85_fut.shp",
                                     join_field, "KEEP_ALL")

arcpy.conversion.ExportFeatures(clim_join, clim_join_path_4)



#Join based on code, this is the unique school identifier
in_field = "FID" 
#join field
join_field = "FID"
clim_join_path_5 = r"join/large_files.gdb/schools_buildings_erased_canopy_clim5"
#run the join which will add directly to school_pts_edge
clim_join = arcpy.management.AddJoin(clim_join_path_4, in_field,
                                     r"climate_models\rcp85_mid.shp",
                                     join_field, "KEEP_ALL")

arcpy.conversion.ExportFeatures(clim_join, clim_join_path_5)

#now we have a file with the canopy and climate information, called clim_join_path_5


## Join to CalEnviroScreen data to the school data file (polygons with erased buildings, canopy  and climate)


In [ ]:
#join one to one or one to many?
join_operation = 'JOIN_ONE_TO_ONE'
#keep all or keep common features
join_type = 'KEEP_ALL'
#field mapping, ignore, '#'
#match option is within a distance for search radius
match_option ='INTERSECT'



schools_final_all_obj = arcpy.analysis.SpatialJoin(clim_join_path_5, cal_enviro, 
                           schools_final_all, 
                           join_operation, join_type,
                           '#', 
                           match_option)

## Join the tables from climate and canopy to a final clean table.  

#### First, manually export the data from schools_final_all_obj as a csv file (not spatial). Load that path into csv_file below

#### Select only the desired columns for the final table

In [ ]:
import pandas as pd

# Specify the file path to the CSV file
csv_file = r"PATH TO YOUR CSV FILE\school_polys_all_data_table.csv"  # Update with your file path

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file,low_memory=False)

subset_df = df[['CDSCode','Status','Level','NCESSCH','SURVYEAR','LEA_NAME','SCH_NAME','LSTREET1','LCITY','LZIP','PHONE','TOTFRL','TOTAL','AM','HI','BL','WH','HP','TR','FTE','ULOCALE','NMCNTY','STUTERATIO','AMALM','AMALF','ASALM','ASALF','HIALM','HIALF','BLALM','BLALF','WHALM','WHALF','HPALM','HPALF','TRALM','TRALF','TOTMENROL','TOTFENROL','AS_','School','TotPop19','CIscore','CIscoreP','Ozone','OzoneP','PM2_5','PM2_5_P','DieselPM','DieselPM_P','Pesticide','PesticideP','Tox_Rel','Tox_Rel_P','Traffic','TrafficP','DrinkWatP','Lead','Lead_P','Cleanup','CleanupP','GWThreat','GWThreatP','HazWaste','HazWasteP','ImpWatBod','ImpWatBodP','PollBurd','PolBurdSc','PolBurdP','Asthma','AsthmaP','LowBirtWt','LowBirWP','Cardiovas','CardiovasP','Educatn','EducatP','Ling_Isol','Ling_IsolP','Poverty','PovertyP','Unempl','UnemplP','HousBurd','HousBurdP','PopChar','PopCharSc','PopCharP','Child_10','Hispanic','White','AfricanAm','NativeAm','OtherMult','SUM','avg30y_h','rcp45_fut','rcp45_mid','rcp85_fut','rcp85_mid']]

#get the correct ID, get correct index
subset_df['OBJECTID']=subset_df.index + 1

#Create Index which ranks schools

#First, normalize the mid rcp 85 values from a scale of 0-100
subset_df['rcp85mid_N']= (subset_df['rcp85_mid']-(min(subset_df['rcp85_mid'])))/((max(subset_df['rcp85_mid']))-(min(subset_df['rcp85_mid'])))*100

#Then, combine calenviroscreen score with normalized climate data
subset_df['scl_rnk']= ((subset_df['rcp85mid_N']) * (0.5)) + ((subset_df['CIscoreP']) *  (0.5))

#save the csv that is clean somewhere new!
subset_df.to_csv(r'\schools\final\school_polys_all_data_table_subset.csv', index=False)

### Join tabular data back to file with school polygons (with no extra data) to create final file

In [ ]:
#Set path to final csv
subset_df = r'\schools\final\school_polys_all_data_table_subset.csv'

#join to copy of the final file that has all the columns deleted except those selected

#reads in path of no data school polygons, which was created manually
schools_no_data = r"schools\final\final.gdb\school_polys_all_data_no_columns"

#Join final file!
join_schools_subset = arcpy.management.AddJoin(schools_no_data, "OBJECTID",
                         subset_df, "OBJECTID", "KEEP_ALL")

#Save it!
final_file = 'YOUR PATH TO FINAL SPATIAL FILE.shp'
arcpy.conversion.ExportFeatures(join_schools_subset, final_file)